In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/processed/cleaned_activities.csv")
df.head()


,account_id,SourceSystem,activity_date,who_id,opportunity_id,opportunity_stage,is_lead,types,Country,solution,activity_order,action_weight
0,0010L00001hVmFhQAK,SFDC_US,2022-07-25,0030L00001vlbHLQAY,NaN,no_opp,1,Follow Up,US,MRS,1.0,1.0
1,0010L00001hVmFhQAK,SFDC_US,2023-02-08,0034X00002xZlQtQAK,NaN,no_opp,1,Call,US,MRS,2.0,1.0
2,0010L00001hVmFhQAK,SFDC_US,2023-02-14,0030L00001vlbHLQAY,NaN,no_opp,1,Call,US,MRS,3.0,1.0
3,0010L00001hVmFhQAK,SFDC_US,2023-02-20,0030L00001vlbHLQAY,NaN,no_opp,2,Call,US,MRS,4.0,1.0
4,0010L00001hVmFhQAK,SFDC_US,2023-03-16,0034X00003GOUrFQAX,NaN,no_opp,1,Call,US,MRS,5.0,1.0


In [2]:
df = df.sort_values(["account_id", "activity_order"])
df["next_action"] = df.groupby("account_id")["types"].shift(-1)
df[["account_id", "types", "next_action", "activity_order"]].head(15)



,account_id,types,next_action,activity_order
0,0010L00001hVmFhQAK,Follow Up,Call,1.0
1,0010L00001hVmFhQAK,Call,Call,2.0
2,0010L00001hVmFhQAK,Call,Call,3.0
3,0010L00001hVmFhQAK,Call,Call,4.0
4,0010L00001hVmFhQAK,Call,Call,5.0
5,0010L00001hVmFhQAK,Call,Call,6.0
6,0010L00001hVmFhQAK,Call,Call,7.0
7,0010L00001hVmFhQAK,Call,Call,8.0
8,0010L00001hVmFhQAK,Call,Call,9.0
9,0010L00001hVmFhQAK,Call,Call,10.0


In [3]:
model_df = df[df["next_action"].notna()].copy()
model_df.shape



(161395, 13)

In [4]:
X = model_df[["Country", "solution", "types", "activity_order"]]
y = model_df["next_action"]

X.head(), y.head()


(  Country solution      types  activity_order
 0      US      MRS  Follow Up             1.0
 1      US      MRS       Call             2.0
 2      US      MRS       Call             3.0
 3      US      MRS       Call             4.0
 4      US      MRS       Call             5.0,
 0    Call
 1    Call
 2    Call
 3    Call
 4    Call
 Name: next_action, dtype: object)

In [5]:
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded.head()


,activity_order,Country_BE,Country_CA,Country_CH,Country_DE,Country_Denmark,Country_FR,Country_Guadeloupe,Country_Guyane Franأ§aise,Country_IE,...,types_2Nd Appointment,types_Call,types_Demo,types_Discovery,types_Email,types_Follow Up,types_Meeting,types_On-Site,types_Review,types_Unknown
0,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,2.0,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,3.0,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,4.0,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,5.0,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((129116, 31), (32279, 31))

In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(
    max_depth=6,
    min_samples_leaf=50,
    random_state=42
)

dt.fit(X_train, y_train)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",6
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",50
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current n

In [8]:
from sklearn.metrics import accuracy_score

y_pred = dt.predict(X_test)
acc = accuracy_score(y_test, y_pred)
acc


0.5847764800644382

In [9]:
feature_importance = (
    pd.DataFrame({
        "feature": X_encoded.columns,
        "importance": dt.feature_importances_
    })
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

feature_importance.head(15)


,feature,importance
0,types_Call,0.412391
1,types_Email,0.289634
2,types_Meeting,0.084510
3,Country_FR,0.056680
4,Country_UK,0.050506
5,activity_order,0.038338
6,types_On-Site,0.028087
7,Country_US,0.017360
8,types_Follow Up,0.014783
9,types_2Nd Appointment,0.003672


In [10]:
def recommend_next_actions(country, solution, current_type, activity_order, model, feature_columns, top_n=4):
    row = pd.DataFrame([{
        "Country": country,
        "solution": solution,
        "types": current_type,
        "activity_order": activity_order
    }])

    row_encoded = pd.get_dummies(row, drop_first=True)
    row_encoded = row_encoded.reindex(columns=feature_columns, fill_value=0)

    probs = model.predict_proba(row_encoded)[0]
    actions = model.classes_

    recs = (
        pd.DataFrame({"action": actions, "probability": probs})
        .sort_values("probability", ascending=False)
        .head(top_n)
        .reset_index(drop=True)
    )
    return recs
#دالة التوصية 

In [11]:
recommend_next_actions(
    country="US",
    solution="MRS",
    current_type="Email",
    activity_order=2,
    model=dt,
    feature_columns=X_encoded.columns,
    top_n=4
)


,action,probability
0,Email,0.347904
1,Meeting,0.152481
2,Call,0.123939
3,Review,0.092826


In [13]:
feature_importance.to_csv("../data/processed/dt_feature_importance.csv", index=False)
